
# Sensitivities: Bump vs AAD 

This dashboard demonstrates the accuracy and performance of AAD sensitivities for a range of complex products
- European Option
- Barrier Option
- Accumulator
- WorstOfBasketSwap
- TaRF
all represented as scripted trades.

We are running ORE twice with a tiny difference in the pricinengine.xml file, 
- setting UseAD to false for the bump run and
- setting UseAD to true for AAD.

## Run with bump & revalue

In [ ]:
from ORE import *
import sys, time, math
sys.path.append('..')
import utilities

In [ ]:
params_bump = Parameters()
params_bump.fromFile("Input/ore_bump.xml")
ore_bump = OREApp(params_bump)
ore_bump.run()
utilities.checkErrorsAndRunTime(ore_bump)
#utilities.writeList(ore_bump.getReportNames())

## Run with AD

In [ ]:
params_ad = Parameters()
params_ad.fromFile("Input/ore_ad.xml")
ore_ad = OREApp(params_ad)
ore_ad.run()
utilities.checkErrorsAndRunTime(ore_ad)
#utilities.writeList(ore_ad.getReportNames())

## Pricing Stats

In [ ]:
pricingstats_bump = ore_bump.getReport("pricingstats")
pricingstats_ad = ore_ad.getReport("pricingstats")
#utilities.checkReportStructure("pricingstats", pricingstats_bump)
print("Bump & Reval Pricing Times (micro seconds):")
display(utilities.format_report(pricingstats_bump))
print("AD Pricing Times:")
display(utilities.format_report(pricingstats_ad))

## NPV reports
We expect identical results.

In [ ]:
npv_bump = ore_bump.getReport("npv")
npv_ad = ore_ad.getReport("npv")
print("Bump & Reval NPV Report:")
display(utilities.format_report(npv_bump))
print("AD NPV Report:")
display(utilities.format_report(npv_ad))

### Sensitivity Reports

In [ ]:
sensi_bump = ore_bump.getReport("sensitivity")
sensi_ad = ore_ad.getReport("sensitivity")
print("Bump:")
display(utilities.format_report(sensi_bump))
print("AD:")
display(utilities.format_report(sensi_ad))

In [ ]:
trade_bump = sensi_bump.dataAsString(0)
factor_bump = sensi_bump.dataAsString(2)
currency_bump = sensi_bump.dataAsString(6)
delta_bump = sensi_bump.dataAsReal(8) 

trade_ad = sensi_ad.dataAsString(0)
factor_ad = sensi_ad.dataAsString(2)
currency_ad = sensi_ad.dataAsString(6)
delta_ad = sensi_ad.dataAsReal(8) 

print ("%-25s %-45s %10s %10s %10s %10s" % ("Trade", "Factor", "Currency", "Delta Bump", "Delta AD", "Diff"))
for i in range(0, sensi_bump.rows()):
    matched = False
    for j in range(0, sensi_ad.rows()):
        if (trade_bump[i] == trade_ad[j] and factor_bump[i] == factor_ad[j]):
            diff = delta_bump[i] - delta_ad[j]
            print("%-25s %-45s %10s %10.2f %10.2f %10.2f" % (trade_bump[i], factor_bump[i], currency_bump[i], delta_bump[i], delta_ad[j], diff))
            matched = True
    if (matched == False):      
        print("%-25s %-45s %10s %10.2f %10s" % (trade_bump[i], factor_bump[i], currency_bump[i], delta_bump[i], "----"))

for j in range(0, sensi_ad.rows()):
    matched = False
    for i in range(0, sensi_bump.rows()):
        if (trade_bump[i] == trade_ad[j] and factor_bump[i] == factor_ad[j]):
            matched = True
    if (matched == False):      
        print("%-25s %-45s %10s %10s %10.2f" % (trade_ad[j], factor_ad[j], currency_ad[j], "----", delta_ad[j]))
